# APT Groups & Malware Attribution Project
## S5115232
### Bournemouth University MSc Internet of Things with Cyber Security Project

## Costas's Code
Code meant to sort and clean our original dataset. Code was originally sent as a python file but disassembled and reassembled to work with Kaggle. This data is now been split into several other dataframes but unsure whether the analysis work below requires any of the variables that were defined in this work; cleandt, classification, etc.

In [73]:
import itertools
from tqdm.auto import tqdm
import pandas as pd
df=pd.read_csv('../input/apt-analysis/dataset.csv',engine="c")
dataset=df.to_dict('records')
df.shape

In [74]:
cleandt=[]
for index, row in df.iterrows():
    tags=eval(row["tags"])
    tgs=[t["tag"] for t in tags]
    cleandt.append({"sha256":row["sha256"],"tags":tgs})

In [75]:
with open("../input/apt-analysis/apt_tags.csv") as fin:
    apts_tags=fin.readlines()
    apts_tags=[t.strip() for t in apts_tags]
classification={}
for t in apts_tags:
    classification[t]=[]
    for s in cleandt:
        if t in s["tags"]:
            classification[t].append(s["sha256"])

In [76]:
for c in classification.keys():
    print(c,len(classification[c]))

# Imports

In [77]:
import enisautility as util
import plotly.graph_objects as go
import pandas as pd
import seaborn as sns
from functools import reduce
import numpy as np
import plotly.express as px
#import tldextract
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

df_triage_samples = pd.read_csv('../input/apt-analysis/triage_samples_key.csv') #Used later with other Triage data sets
modified_df = pd.read_csv('../input/apt-analysis/modified_dataset.csv')

***
## Dictionary and List Conversion

In [78]:
classification_df = pd.DataFrame.from_dict({k:pd.Series(v) for k,v in classification.items()},orient='index') #https://stackoverflow.com/questions/52302645/convert-dictionary-unequal-items-to-data-frame
cleandt_df = pd.DataFrame(cleandt, columns=['sha256', 'tags']) 

#transposed_df = classification_df.transpose()
#cleandt #Entire Dataset for APTs
#classification #APT Groups & Samples

***
# Unassigned CSVs

In [79]:
#df_mal = pd.read_csv('../input/apt-analysis/VT_malicious.csv')
#df_tsub = pd.read_csv('../input/apt-analysis/VT_times_submitted.csv')
#df_und = pd.read_csv('../input/apt-analysis/VT_undetected.csv')


#df_triage_credentials = pd.read_csv('../input/apt-analysis/triage_credentials.csv')
#df_triage_ioc = pd.read_csv('../input/apt-analysis/triage_iocs_ips.csv') #Slightly Broken

***
## 1.0 Group's Format Speciality 

In [80]:
df_vt_tdesc = pd.read_csv('../input/apt-analysis/VT_type_description.csv')
df_vt_text = pd.read_csv('../input/apt-analysis/VT_type_extension.csv')
df_vt_tyta = pd.read_csv('../input/apt-analysis/VT_type_tag.csv')
df_vt_mag = pd.read_csv('../input/apt-analysis/VT_magic.csv')

format_data_frames = [modified_df, df_vt_tdesc, df_vt_text, df_vt_tyta, df_vt_mag]

df_vt_format = reduce(lambda  left,right: pd.merge(left,right,on=['sha256'],
                                            how='outer'), format_data_frames).fillna('NaN')

df_vt_format = df_vt_format[df_vt_format['apt_group']!='NaN']
df_vt_format

In [81]:
top10_types = df_vt_format.type_description.groupby(df_vt_format.type_description).count().sort_values(ascending=False).head(10)
top10_types = top10_types.to_frame()
top10_types.rename(columns={'type_description': 'type', 'type_description': 'c'}, inplace=True)

top10_types.columns=['c']
top10_types.reset_index(level=0, inplace=True)
top10_types.columns=['type','c']
fig = go.Figure()
fig.add_bar(x=top10_types["type"], y=top10_types['c'])

fig.update_layout(title='Top 10 File Types',
                  xaxis = dict(title='File Type', type='category'),
                  yaxis = dict(title='Total Occurrence'),width=600)

In [82]:
fig = px.pie(top10_types, values="c", names="type")
fig.show()

***
## 2.0 Threat Category

In [83]:
df_vt_stl = pd.read_csv('../input/apt-analysis/VT_suggested_threat_label.csv')

df_vt_threatlabel = modified_df.merge(df_vt_stl, how='inner', on='sha256')
df_vt_threatlabel

In [84]:
to_plot = df_vt_threatlabel['suggested_threat_label'].value_counts().nlargest(11)

fig = go.Figure()
fig.add_bar(x=to_plot.index, y=to_plot.values)

fig.update_layout(title='Top 10 Suggested Threat Label',
                  xaxis = dict(title='Suggested Threat Label', type='category'),
                  yaxis = dict(title='Total Occurrence'),width=600)

### 2.1 STL

In [85]:
df_vt_threatlabel['suggested_threat_label'] = df_vt_threatlabel['suggested_threat_label'].str.split('.').str[0]
df_vt_threatlabel

In [86]:
to_plot = df_vt_threatlabel['suggested_threat_label'].value_counts().nlargest(10)

fig = go.Figure()
fig.add_bar(x=to_plot.index, y=to_plot.values)

fig.update_layout(title='Threat Category',
                  xaxis = dict(title='Category', type='category'),
                  yaxis = dict(title='Total Occurrence'),width=600)

***
## 3.0 Timeline

In [87]:
df_vt_fsitw = pd.read_csv('../input/apt-analysis/VT_first_seen_itw_date.csv')
df_vt_fsitw['first_seen_itw_date'] = pd.to_datetime(df_vt_fsitw['first_seen_itw_date'],unit='s')

df_vt_fsd = pd.read_csv('../input/apt-analysis/VT_first_submission_date.csv')
df_vt_fsd['first_submission_date'] = pd.to_datetime(df_vt_fsd['first_submission_date'],unit='s')

df_vt_lad = pd.read_csv('../input/apt-analysis/VT_last_analysis_date.csv')
df_vt_lad['last_analysis_date'] = pd.to_datetime(df_vt_lad['last_analysis_date'],unit='s')

df_vt_lmd = pd.read_csv('../input/apt-analysis/VT_last_modification_date.csv')
df_vt_lmd['last_modification_date'] = pd.to_datetime(df_vt_lmd['last_modification_date'],unit='s')

data_frames = [modified_df, df_vt_fsitw, df_vt_fsd, df_vt_lad, df_vt_lmd]

df_dates = reduce(lambda  left,right: pd.merge(left,right,on=['sha256'],
                                            how='outer'), data_frames).fillna('NaN')

df_dates

### 3.1 Differences

In [88]:
df_timedifferences  = df_vt_fsitw.merge(df_vt_fsd, how='inner', on='sha256')
df_timedifferences = modified_df.merge(df_timedifferences, how='inner', on='sha256')
df_timedifferences['Number_of_days'] = ((df_timedifferences.first_submission_date - df_timedifferences.first_seen_itw_date)/np.timedelta64(1, 'D'))
df_timedifferences['Number_of_days'] = df_timedifferences['Number_of_days'].astype(int)
df_timedifferences = df_timedifferences[df_timedifferences['Number_of_days'] > 0]

discard = ["apt20"]
df_timedifferences = df_timedifferences[~df_timedifferences.apt_group.str.contains('|'.join(discard))] #Drops yearly APT groups
df_timedifferences

In [89]:
df_difference = df_timedifferences.groupby(by=['apt_group'])['Number_of_days'].mean() #Idea was to add up the days of all differences, hopefully this can speak to capability if malware goes long undetected
df_difference = df_difference.to_frame()
df_difference.sort_values(by=['Number_of_days'], inplace=True, ascending=False)
df_difference = df_difference.rename_axis('apt_group').reset_index()
df_difference

In [90]:
fig = px.bar(df_difference, x='apt_group', y='Number_of_days')

fig.update_layout(title='Average number of days between First-Seen-In-The-Wild and First Submitted.',
                  xaxis = dict(title='APT Group', type='category'),
                  yaxis = dict(title='Days'),width=600)
fig.show()


The idea behind this was to attempt to show an APT group's capability based on how long it took for their malware to be first seen until it was first submitted. The total days for each malware attributed to a group were added and averaged to provide the above data. unfortunately the idea was more interesting than the execution. These numbers can be based on several factors: little samples from the same attack, a large amount of samples, etc.  

***
## 4.0 YARA Rules

In [91]:
df_vx_tag = pd.read_csv('../input/apt-analysis/vx_tag.csv')
df_vx_tag = modified_df.merge(df_vx_tag, how='inner', on='sha256')
df_vx_yara = df_vx_tag[df_vx_tag['tag'].str.contains('yara').fillna(False)]
df_vx_yara

In [92]:
top10_yara = df_vx_yara.tag.groupby(df_vx_yara.tag).count().sort_values(ascending=False).head(10)
top10_yara = top10_yara.to_frame()
top10_yara.rename(columns={'tag': 'tag', 'tag': 'c'}, inplace=True)

top10_yara.columns=['c']
top10_yara.reset_index(level=0, inplace=True)
top10_yara.columns=['tag','c']
fig = go.Figure()
fig.add_bar(x=top10_yara["tag"], y=top10_yara['c'])

fig.update_layout(title='Top 10 YARA Rules',
                  xaxis = dict(title='YARA Rule', type='category'),
                  yaxis = dict(title='Total Occurrence'),width=600)

***
### 4.1 Cherrypicked YARA Rules

In [93]:
search_values = ['yara:seh_init','yara:seh_save','yara:anti_dbg','yara:ispacked','yara:screenshot','yara:keylogger',
                 'yara:hasdigitalsignature','yara:escalate_priv','yara:win_hook','yara:maldoc_find_kernel32_base_method_1']
df_cp_yara = df_vx_yara[df_vx_yara.tag.str.contains('|'.join(search_values ))]

df_cp_yara

In [94]:
df_cp_yara_c = df_cp_yara.tag.groupby(df_cp_yara.tag).count().sort_values(ascending=False)
df_cp_yara_c = df_cp_yara_c.to_frame()
df_cp_yara_c.rename(columns={'tag': 'tag', 'tag': 'c'}, inplace=True)

df_cp_yara_c.columns=['c']
df_cp_yara_c.reset_index(level=0, inplace=True)
df_cp_yara_c.columns=['tag','c']
fig = go.Figure()
fig.add_bar(x=df_cp_yara_c["tag"], y=df_cp_yara_c['c'])

fig.update_layout(title='Interesting YARA Rules',
                  xaxis = dict(title='YARA Rule', type='category'),
                  yaxis = dict(title='Total Occurrence'),width=500)

***
## 5.0 IPs & Domains
Both VX-Underground & Triage C2 servers have been combined into a single CSV file.

In [95]:
df_c2 = pd.read_csv('../input/apt-analysis/c2.csv')
df_c2 = modified_df.merge(df_c2, how='outer', on='sha256')
df_c2 = util.tidy_split(df_c2,column='c2',sep=',')
df_c2 = df_c2[df_c2['c2']!='nan']
df_c2 = df_c2[df_c2['apt_group'].notna()]
df_c2

In [96]:
#df_c2['Domain'] = df_c2['c2'].apply(lambda url: tldextract.extract(url).domain)
#df_c2

***
## 6.0 ATT&CK

In [97]:
df_triage_attack = pd.read_csv('../input/apt-analysis/triage_attack.csv')
df_triage_attack = util.tidy_split(df_triage_attack,column='attack_id',sep=',')
df_triage_attack = df_triage_attack[df_triage_attack['attack_id']!='nan']
df_triage_attack = modified_df.merge(df_triage_attack, how='inner', on='sha256')
df_triage_attack

In [98]:
top10_attack = df_triage_attack.attack_id.groupby(df_triage_attack.attack_id).count().sort_values(ascending=False).head(10)
top10_attack = top10_attack.to_frame()
top10_attack.rename(columns={'attack_id': 'attack_id', 'attack_id': 'c'}, inplace=True)

top10_attack.columns=['c']
top10_attack.reset_index(level=0, inplace=True)
top10_attack.columns=['attack_id','c']
fig = go.Figure()
fig.add_bar(x=top10_attack["attack_id"], y=top10_attack['c'])

fig.update_layout(title='Top 10 ATTACK Techniques',
                  xaxis = dict(title='Technique', type='category'),
                  yaxis = dict(title='Total Occurrence'))

In [99]:
top10_attack

The top 3 techniques associated with vulnerabilities are:
* [T1082 – System Information Discovery](https://attack.mitre.org/techniques/T1082/) (23,983 Techniques Associated) 
* [T1112 – Modify Registry](https://attack.mitre.org/techniques/T1112/) (16,664 Techniques Associated) 
* [T1012 – Query Registry](https://attack.mitre.org/techniques/T1012/) (15,388 Techniques Associated) 

***
## 7.0 Malware Families

In [100]:
df_family = pd.read_csv('../input/apt-analysis/malware_families.csv')
df_family

In [101]:
top10_family = df_family.family.groupby(df_family.family).count().sort_values(ascending=False).head(10)
top10_family = top10_family.to_frame()
top10_family.rename(columns={'family': 'family', 'family': 'c'}, inplace=True)

top10_family.columns=['c']
top10_family.reset_index(level=0, inplace=True)
top10_family.columns=['family','c']
fig = go.Figure()
fig.add_bar(x=top10_family["family"], y=top10_family['c'])

fig.update_layout(title='Top 10 Malware Families',
                  xaxis = dict(title='Malware Family', type='category'),
                  yaxis = dict(title='Total Occurrences'))

***
# Clustering
These are the imports for the clustering work that we will hopefully be doing after the submission of the MSc thesis.

In [102]:
df_vt_vhash = pd.read_csv('../input/apt-analysis/VT_vhash.csv')
df_vt_ssd = pd.read_csv('../input/apt-analysis/VT_ssdeep.csv')
df_vt_tlsh = pd.read_csv('../input/apt-analysis/VT_tlsh.csv')
df_vx_crc32 = pd.read_csv('../input/apt-analysis/vx_crc32.csv')

### Imphash

In [103]:
df_imphash = pd.read_csv('../input/apt-analysis/imphash.csv')
df_imphash